In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Endometrioid_Cancer/GSE109058'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A unique pair of microarray datasets for microRNA profiling: data without careful design (Dataset B)"
!Series_summary	"We set out to demonstrate the logistic feasibility of careful study design in microarray studies and the level of scientific benefits it can provide, in comparison with post-hoc data adjustment such as normalization, for the purpose of reducing the impact of handling effects on the accuracy and reproducibility of microarray data analysis. Towards this end, we conducted a study of microRNA expression using endometroid endometrial cancer tumors (n=96) and serous ovarian cancer tumors (n=96) that were all primary, untreated, and collected in 2000-2012 at Memorial Sloan Kettering Cancer Center. The same set of tumor tissue samples were profiled twice using the Agilent microRNA microarrays (Human miRNA V16.0), once with uniform handling and careful design for the allocation of arrays to samples and the second study without."
!Series_ov

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
if 'miRNA' not in "!Series_title".lower() and 'miRNA' not in "!Series_summary".lower():
    is_gene_available = True

# Variable availability:
sample_characteristics_dict = {
    0: ['disease state: endometrioid endometrial cancer', 'disease state: advanced serous ovarian cancer'],
    1: ['tissue: tumor'],
    2: ['batch: BATCH1', 'batch: BATCH2', 'batch: BATCH3', 'batch: BATCH4', 'batch: BATCH5']
}

# Endometrioid_Cancer trait
trait_key_possible_values = sample_characteristics_dict.get(0, [])
if len(set(trait_key_possible_values)) > 1:
    trait_row = 0
else:
    trait_row = None

# Age data is not available in the parsed characteristics data
age_row = None

# Gender data is not available in the parsed characteristics data
gender_row = None

# Data type conversion functions
def convert_trait(value):
    value = value.split(':')[1].strip()
    if value == 'endometrioid endometrial cancer':
        return 1
    elif value == 'advanced serous ovarian cancer':
        return 0
    else:
        return None

# For age and gender, since they are not available, the functions will just return None
convert_age = lambda value: None
convert_gender = lambda value: None

# Save cohort information
save_cohort_info('GSE109058', './preprocessed/Endometrioid_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Endometrioid_Cancer', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Endometrioid_Cancer/trait_data/GSE109058.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM2928821': [1], 'GSM2928822': [1], 'GSM2928823': [1], 'GSM2928824': [1], 'GSM2928825': [1], 'GSM2928826': [0], 'GSM2928827': [1], 'GSM2928828': [1], 'GSM2928829': [1], 'GSM2928830': [0], 'GSM2928831': [1], 'GSM2928832': [1], 'GSM2928833': [1], 'GSM2928834': [1], 'GSM2928835': [1], 'GSM2928836': [1], 'GSM2928837': [1], 'GSM2928838': [0], 'GSM2928839': [1], 'GSM2928840': [1], 'GSM2928841': [1], 'GSM2928842': [0], 'GSM2928843': [1], 'GSM2928844': [1], 'GSM2928845': [1], 'GSM2928846': [0], 'GSM2928847': [1], 'GSM2928848': [0], 'GSM2928849': [1], 'GSM2928850': [1], 'GSM2928851': [1], 'GSM2928852': [1], 'GSM2928853': [1], 'GSM2928854': [1], 'GSM2928855': [0], 'GSM2928856': [1], 'GSM2928857': [1], 'GSM2928858': [1], 'GSM2928859': [1], 'GSM2928860': [1], 'GSM2928861': [1], 'GSM2928862': [1], 'GSM2928863': [1], 'GSM2928864': [1], 'GSM2928865': [0], 'GSM2928866': [1], 'GSM2928867': [1], 'GSM2928868': [1], 'GSM2928869': [1], 'GSM2928870': [1], 'GSM2928871': [1], 'GSM2928872': [1], 'GSM2928873

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['(-)3xSLv1', 'A_25_P00010019', 'A_25_P00010020', 'A_25_P00010021',
       'A_25_P00010023', 'A_25_P00010041', 'A_25_P00010042', 'A_25_P00010043',
       'A_25_P00010044', 'A_25_P00010047', 'A_25_P00010048', 'A_25_P00010053',
       'A_25_P00010054', 'A_25_P00010062', 'A_25_P00010063', 'A_25_P00010070',
       'A_25_P00010071', 'A_25_P00010072', 'A_25_P00010073', 'A_25_P00010078'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_25_P00010019', 'A_25_P00010020', 'A_25_P00010021', 'A_25_P00010023', 'A_25_P00010041'], 'miRNA_ID': ['hsa-miR-329', 'hsa-miR-329', 'hsa-miR-655', 'hsa-miR-369-3p', 'hsa-miR-363*'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'GENE_SYMBOL': ['hsa-miR-329', 'hsa-miR-329', 'hsa-miR-655', 'hsa-miR-369-3p', 'hsa-miR-363*'], 'ACCESSION_STRING': ['mir|hsa-miR-329|mir|MIMAT0001629', 'mir|hsa-miR-329|mir|MIMAT0001629', 'mir|hsa-miR-655|mir|MIMAT0003331', 'mir|hsa-miR-369-3p|mir|MIMAT0000721', 'mir|hsa-miR-363*|mir|MIMAT0003385'], 'CHROMOSOMAL_LOCATION': ['chr14:101493173-101493192,chr14:101493490-101493509', 'chr14:101493174-101493192,chr14:101493491-101493509', 'chr14:101515949-101515968', 'chr14:101531979-101531998', 'chrX:133303435-133303418'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'GENE_SYMBOL'

# 2. Get the dataframe storing the mapping between probe IDs and genes
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and get the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, gene_mapping)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Endometrioid_Cancer/gene_data/GSE109058.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Endometrioid_Cancer')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE109058', './preprocessed/Endometrioid_Cancer/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Endometrioid_Cancer/GSE109058.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
